<a href="https://colab.research.google.com/github/yds04312/nownow/blob/master/yolo_training_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect COLAB with Google Drive

이걸로 프로그래밍 돌리기!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [3]:
#이거다 이거
#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
#!apt-get update
!apt-get install cuda-10-0

--2021-05-12 03:27:00--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940 (2.9K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’

cuda-repo-ubuntu180 100%[===================>]   2.87K  --.-KB/s    in 0s      

2021-05-12 03:27:00 (149 MB/s) - ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’ saved [2940/2940]

Selecting previously unselected package cuda-repo-ubuntu1804.
(Reading database ... 123778 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1804_10.0.130-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1804 (10.0.130-1) ...
Setting up cuda-repo-ubuntu1804 (10.0.130-1) ...

Configuration file '/etc/apt/sources.li

# Check your GPU compatibility with CUDA, as you can see COLAB GPU has CUDA 10.0 installed

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


# Load dataset and darknet folder to COLAB working space by unzip file "darknet.zip"

 * Use syntax !unzip + <path_to_darknet.zip_in_your_GDrive> to extract folder darknet. In my case, "darknet.zip" locates in My Drive/Colab Notebooks/yolo
 

In [5]:
!unzip "/content/drive/MyDrive/Colab Notebooks/yolo/darknet.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/yolo/darknet.zip
   creating: darknet/
  inflating: darknet/.gitignore      
   creating: darknet/cfg/
  inflating: darknet/cfg/alexnet.cfg  
  inflating: darknet/cfg/cifar.cfg   
  inflating: darknet/cfg/cifar.test.cfg  
  inflating: darknet/cfg/coco.data   
  inflating: darknet/cfg/combine9k.data  
  inflating: darknet/cfg/darknet.cfg  
  inflating: darknet/cfg/darknet19.cfg  
  inflating: darknet/cfg/darknet19_448.cfg  
  inflating: darknet/cfg/darknet53.cfg  
  inflating: darknet/cfg/darknet53_448.cfg  
  inflating: darknet/cfg/darknet9000.cfg  
  inflating: darknet/cfg/densenet201.cfg  
  inflating: darknet/cfg/extraction.cfg  
  inflating: darknet/cfg/extraction.conv.cfg  
  inflating: darknet/cfg/extraction22k.cfg  
  inflating: darknet/cfg/go.cfg      
  inflating: darknet/cfg/go.test.cfg  
  inflating: darknet/cfg/gru.cfg     
  inflating: darknet/cfg/imagenet1k.data  
  inflating: darknet/cfg/imagenet22k.dataset  
  inflating: 


#Compile darknet directory with below script:

In [6]:
%cd /content/darknet
!make
!chmod +x ./darknet

/content/darknet
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:947:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/

#Save weight during training in your Google Drive

This step is important since COLAB environment will be recycle after 12 hours and all files locate in its working space will be deleted. Here we defines a symbolic link to save the weight directly into our backup folder which we created in our GDrive before. In my case, my backup folder directory is My Drive/YOLOv3_weight/backup.

In [7]:
!rm /content/darknet/backup -r
!ln -s /content/drive/MyDrive/YOLOv3_weight/backup /content/darknet

Install dos2unix to convert train.txt, val.txt, yolo.data, yolo.names, yolov3_custom_train.cfg to unix

In [8]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (346 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 137070 files and directories curren

In [9]:
!dos2unix /content/darknet/data/train.txt 
!dos2unix /content/darknet/data/val.txt    #./darknet/data/val.txt
!dos2unix /content/darknet/data/yolo.data
!dos2unix /content/darknet/data/yolo.names
!dos2unix /content/darknet/cfg/yolov3_custom_train.cfg

dos2unix: converting file /content/darknet/data/train.txt to Unix format...
dos2unix: converting file /content/darknet/data/val.txt to Unix format...
dos2unix: converting file /content/darknet/data/yolo.data to Unix format...
dos2unix: converting file /content/darknet/data/yolo.names to Unix format...
dos2unix: converting file /content/darknet/cfg/yolov3_custom_train.cfg to Unix format...


In [10]:
%cd /content/darknet
!./darknet detector train /content/darknet/data/yolo.data  /content/darknet/cfg/yolov3_custom_train.cfg

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.015947, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.181789, Class: 0.379192, Obj: 0.071418, No Obj: 0.105962, .5R: 0.250000, .75R: 0.000000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.041815, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.016331, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.434621, Class: 0.467049, Obj: 0.118135, No Obj: 0.104355, .5R: 0.250000, .75R: 0.250000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.040826, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.015485, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.121519, Class: 0.645955, Obj: 0.123201, No Obj: 0.102630, .5R: 0.000000, .75R: 0.000000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.041258, .5R: -nan, .75R: -nan,  count: 0
R

In [11]:
#%cd /content/darknet
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg /content/drive/MyDrive/YOLOv3_weight/backup/yolov3_custom_train_200.weights -dont_show -map

#기존에 끊긴 연결 다시하기

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.814562, Class: 0.999309, Obj: 0.771408, No Obj: 0.005548, .5R: 1.000000, .75R: 1.000000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000002, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.739787, Class: 0.999611, Obj: 0.935494, No Obj: 0.004747, .5R: 1.000000, .75R: 0.750000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000002, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.801885, Class: 0.999188, Obj: 0.776742, No Obj: 0.004616, .5R: 1.000000, .75R: 0.750000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000002, .5R: -nan, .75R: -nan,  count: 0
R

In [19]:
!./darknet detector test data/yolo.data cfg/yolov3_custom_train.cfg /content/drive/MyDrive/YOLOv3_weight/backup/yolov3_custom_train_final.weights  /content/drive/MyDrive/aug_data/chinchin.JPG -thresh 0.2


layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 